# BASICS 03 - HOTS with online classification (Grimaldi et al. 2022)

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%cd ../hots
import tonic, torch, os, pickle
from tqdm import tqdm
from network import network
from layer import mlrlayer
from timesurface import timesurface
from utils import get_loader, make_histogram_classification, HOTS_Dataset, fit_mlr, predict_mlr, score_classif_events

print(f'Tonic version installed -> {tonic.__version__}')

print(f'Number of GPU devices available: {torch.cuda.device_count()}')
for N_gpu in range(torch.cuda.device_count()):
    print(f'GPU {N_gpu+1} named {torch.cuda.get_device_name(N_gpu)}')

/home/antoine/homhots/hotsline/hots
Tonic version installed -> 1.0.19
Number of GPU devices available: 1
GPU 1 named GeForce RTX 2080 Ti


## Loading of the dataset for the clustering phase

In [3]:
type_transform = tonic.transforms.NumpyAsType(int)
trainset = tonic.datasets.POKERDVS(save_to='../../Data/', train=True, transform=type_transform)
testset = tonic.datasets.POKERDVS(save_to='../../Data/', train=False, transform=type_transform)
loader = get_loader(trainset)
num_sample_train = len(loader)
num_sample_test = len(testset)
print(f'number of samples in the training set: {len(loader)}')

number of samples in the training set: 48


## Initialization of the network

In [4]:
name = 'homeohots'
homeo = True
timestr = '2022-04-27'
dataset_name = 'poker'

Rz = [2, 4]
N_neuronz = [8, 16]
tauz = [1e3, 4e3]

hots = network(name, dataset_name, timestr, trainset.sensor_size, nb_neurons = N_neuronz, tau = tauz, R = Rz, homeo = homeo)

initial_name = hots.name

device -> cuda


## Unsupervised clustering

In [5]:
if not os.path.exists('../Records/'):
    os.mkdir('../Records/')
    os.mkdir('../Records/networks/')
filtering_threshold = [2*Rz[L] for L in range(len(Rz))]
path = '../Records/networks/'+hots.name+'.pkl'
if not os.path.exists(path):
    hots.clustering(loader, trainset.ordering, filtering_threshold, record = True)

## Training of the classification layer

In [6]:
jitter = (None, None)
num_workers = 0
learning_rate = 0.005
beta1, beta2 = 0.9, 0.999
betas = (beta1, beta2)
num_epochs = 2 ** 5 + 1
N_output_neurons = N_neuronz[-1]
ts_size = (trainset.sensor_size[0],trainset.sensor_size[1],N_output_neurons)
tau_cla = 2e4

train_path = f'../Records/output/train/{hots.name}_{num_sample_train}_{jitter}/'
model_path = f'../Records/networks/{hots.name}_{tau_cla}_{learning_rate}_{betas}_{num_epochs}_{jitter}.pkl'

trainset_output = HOTS_Dataset(train_path, trainset.sensor_size, dtype=trainset.dtype, transform=type_transform)
trainloader = get_loader(trainset_output)
classif_layer, losses = fit_mlr(trainloader, model_path, tau_cla, learning_rate, betas, num_epochs, ts_size, trainset.ordering, len(trainset.classes))

In [ ]:
import numpy as np

std_jit_s = np.linspace(0,20,20)
std_jit_t = np.logspace(2.7,4.2,20)
var_jit_s = std_jit_s**2
nb_trials = 10

score_spatial = []
score_temporal = []

torch.set_default_tensor_type("torch.DoubleTensor")

for trial in range(nb_trials):
    for spatial_jitter in var_jit_s:
        if spatial_jitter==0:
            jitter = (None,None)
        else:
            jitter = (spatial_jitter,None)
        test_path = f'../Records/output/test/{initial_name}_{trial}_{num_sample_test}_{jitter}/'
        results_path = f'../Records/LR_results/{initial_name}_{trial}_{tau_cla}_{learning_rate}_{betas}_{num_epochs}_{jitter}.pkl'
        hots.name = initial_name+f'_{trial}'
        
        spatial_jitter_transform = tonic.transforms.SpatialJitter(sensor_size = trainset.sensor_size, variance_x = spatial_jitter, variance_y = spatial_jitter, clip_outliers = True)
        spatial_jitter_transform_full = tonic.transforms.Compose([spatial_jitter_transform, type_transform])
        testset = tonic.datasets.POKERDVS(save_to='../../Data/', train=False, transform=spatial_jitter_transform_full)
        loader = get_loader(testset)
        hots.coding(loader, testset.ordering, testset.classes, filtering_threshold, training=False, jitter = jitter, verbose=False)
        
        testset_output = HOTS_Dataset(test_path, trainset.sensor_size, dtype=trainset.dtype, transform=type_transform)
        testloader = get_loader(testset_output)
        
        likelihood, true_target, timestamps = predict_mlr(classif_layer,tau_cla,testloader,results_path,ts_size,testset_output.ordering)
        meanac, onlinac, lastac, _, _ = score_classif_events(likelihood, true_target, verbose=False)
        print(f'For jitter variance {spatial_jitter} pixels, accuracy is {lastac*100}%')

For jitter variance 0.0 pixels, accuracy is 100.0%
For jitter variance 1.1080332409972298 pixels, accuracy is 95.0%
For jitter variance 4.432132963988919 pixels, accuracy is 75.0%
For jitter variance 9.97229916897507 pixels, accuracy is 75.0%
For jitter variance 17.728531855955676 pixels, accuracy is 65.0%
For jitter variance 27.700831024930743 pixels, accuracy is 45.0%
For jitter variance 39.88919667590028 pixels, accuracy is 50.0%
For jitter variance 54.29362880886426 pixels, accuracy is 45.0%
For jitter variance 70.9141274238227 pixels, accuracy is 45.0%
For jitter variance 89.75069252077562 pixels, accuracy is 40.0%
For jitter variance 110.80332409972297 pixels, accuracy is 30.0%
For jitter variance 134.0720221606648 pixels, accuracy is 50.0%
For jitter variance 159.55678670360112 pixels, accuracy is 40.0%
For jitter variance 187.25761772853184 pixels, accuracy is 40.0%
For jitter variance 217.17451523545705 pixels, accuracy is 35.0%
For jitter variance 249.30747922437672 pixels, a

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [00:02<00:00,  6.69it/s]


For jitter variance 359.00277008310246 pixels, accuracy is 40.0%


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [00:03<00:00,  6.40it/s]


device -> cuda - num_workers -> 0


...

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [00:14<00:00,  1.42it/s]


device -> cuda - num_workers -> 0


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [00:09<00:00,  2.16it/s]


For jitter variance 0.0 pixels, accuracy is 100.0%


 20%|██████████████████████████████████████████████▊                                                                                                                                                                                           | 4/20 [00:02<00:09,  1.64it/s]

In [ ]:
print(len([]))